In [1]:
#!pip install scikit-learn --force
#!pip install catboost

In [1]:
import sys
sys.path.append('../..')
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd
from datetime import datetime
import os
import importlib

pd.set_option('display.max.columns', 300)

from core.calculator.storage import ModelDB
from core.calculator.core import ForecastConfig
from core.calculator.core import TrainingManager
from core.calculator.core import ForecastConfig
from core.calculator.core import ForecastEngine

from core.calculator.deposits import DepositsCalculationType
from core.calculator.deposits import DepositIterativeCalculator

from core.definitions import *
from core.project_update import load_portfolio

from core.models import DepositModels
from warnings import filterwarnings
filterwarnings('ignore')

## Корректируем таблицу по НС

In [2]:
# для создания табличек

from core.models.utils import run_spark_session
spark = run_spark_session('create_table_tnd_v2')

#spark = None #если без обучения

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/22 18:44:23 WARN ipc.Client: Exception encountered while connecting to the server : org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.ipc.StandbyException): Operation category READ is not supported in state standby. Visit https://s.apache.org/sbnn-error
23/10/22 18:44:24 WARN ipc.Client: Exception encountered while connecting to the server : org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.ipc.StandbyException): Operation category READ is not supported in state standby. Visit https://s.apache.org/sbnn-error
23/10/22 18:44:25 WARN yarn.Client: Same path resource file:///opt/cloudera/parcels/AnacondaPy37/jars/spark-tree-plotting-0.2.jar added multiple times to distributed cache.
23/10/22 18:44:38 WARN ipc.Client: Exception encountered while connecting to the server : org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.ipc.StandbyExceptio

In [3]:
query2 = """
with cte 
as
(
SELECT 				
    CAST (last_day(foa.ENTRY_DT) AS TIMESTAMP) AS REPORT_DT,				
    CAST (CASE WHEN product_bp_group_nm = 'Накопительные счета' THEN 'PR_SAAC' 				
            WHEN product_bp_group_nm in ('Дебетовые Карты', 'Текущие счета') THEN 'PR_TC' 				
            ELSE 'PR_DEP' END as varchar(50)) as PROD_smot,				
                    				
    CASE 				
                WHEN FOA.PRODUCT_BP_GROUP_NM LIKE 'Депозиты%' THEN 'Депозиты'				
                WHEN FOA.PRODUCT_BP_CD = '300034' THEN 'Escrow' 				
                WHEN FOA.PRODUCT_BP_CD IN ('303003', '393002') OR B.DEPOSIT_TYPE_CD = 'SA' THEN 'Накопительные'		
				WHEN foa.PRODUCT_BP_GROUP_NM = 'Накопительные счета' THEN 'Накопительные'
                WHEN SUBSTR(B.ACCOUNT_NUM,1,5) IN ('40901','40902') THEN 'Аккредитивы'				
                WHEN SUBSTR(B.ACCOUNT_NUM,1,5) = '52206' THEN 'Сберегательные сертификаты'				
                WHEN foa.PRODUCT_BP_GROUP_NM IS NULL THEN 'Депозиты'				
                ELSE 'Текущие' END AS PRODUCT_NM,    				
    FOA.PRODUCT_BANK_GROUP_NM,				
    FOA.PRODUCT_BP_GROUP_NM,				
    CASE WHEN B.CURRENCY_RK = 48 THEN 'RUR'				
                WHEN B.CURRENCY_RK = 60 THEN 'ЕUR'	
				WHEN B.CURRENCY_RK = 8 THEN 'CNY'
				WHEN B.CURRENCY_RK IS NULL THEN 'RUR'
                ELSE 'USD' END AS VAL_3,				
   				
    COALESCE (CUST.SERVICE_MODEL_CD, 'Массовый') AS MODEL,				
    CUST.SERVICE_MODEL_CD AS MODEL_DETAIL,	
                				
(cast(PROFIT_LOSSES_FLG as decimal(2,1))  * foa.ENTRY_RUR_AMT) as losses				
				
				
FROM CDM.FCT_OPERATION_ANALYTICS foa				
--				
LEFT JOIN CDM.CD_DEPOSIT b           				
        ON foa.AGREEMENT_RK=b.AGREEMENT_RK                        				
        AND b.DELETED_FLG='0'                        				
        AND CAST (foa.ENTRY_DT AS TIMESTAMP) BETWEEN b.EFFECTIVE_FROM_DTTM AND b.EFFECTIVE_TO_DTTM   				
--				
LEFT JOIN  CDM.TECH_CUSTOMER_X_GLOBAL AS C				
        ON foa.CUSTOMER_RK = C.CUSTOMER_RK  				
        AND C.CUSTOMER_GLOBAL_TYPE='MDM'				
        AND  C.EFFECTIVE_TO_DTTM = CAST('5999-12-31 00:00:00' AS TIMESTAMP)				
        AND C.DELETED_FLG='0'				
--				
LEFT JOIN CDM.CD_CUSTOMER_SERVICE CUST				
    ON CUST.CUSTOMER_GLOBAL_RK = C.CUSTOMER_GLOBAL_RK				
    AND CAST (last_day(foa.ENTRY_DT) AS TIMESTAMP)  BETWEEN CUST.EFFECTIVE_FROM_DTTM AND CUST.EFFECTIVE_TO_DTTM				
    AND CUST.IS_ACTIVE_FLG<>'2'				
    AND CUST.DELETED_FLG='0'				
--                				
WHERE 1=1				
        AND PNL_REPORT_CD='020'	
		AND foa.PRODUCT_BP_GROUP_NM IN ('Дебетовые Карты', 'Депозиты', 'Накопительные счета', 'Текущие счета')
    --    AND UPPER(foa.PRODUCT_BANK_GROUP_NM) IN ('РКО', 'ДЕПФЛ', 'РКПРОЧ', 'ВИП', 'ДЕБЕТОВЫЕ КАРТЫ') 				
--AND foa.PRODUCT_CORE_FLG = 'Y'				
                        AND foa.ENTRY_DT BETWEEN CAST('2023-01-01' AS TIMESTAMP) AND CAST('2023-12-31' AS TIMESTAMP) 				
                        AND foa.EFFECTIVE_TO_DTTM = CAST('5999-12-31 00:00:00' AS TIMESTAMP)				
                        AND foa.DELETED_FLG='0'				
                        AND IS_SPOD_FLG='0'	
						AND FOA.IS_ACTIVE_FLG = '1'
                        AND foa.PNL_ALGORITHM_CD = 'FOA_DIRECT_INT'
						AND FOA.SOURCE_SYSTEM_CD IN ('00000', '00002', '00006', '00018', '00040', '00051', '00055', '00056', '00072', '95013', '98000', '99995')
--GROUP BY 1,2,3,4,5,6,7,8	

)


select *
from cte
		

"""

In [4]:
df_table = spark.sql(query2)

Py4JJavaError: An error occurred while calling o93.sql.
: org.apache.hadoop.security.AccessControlException: Permission denied: user=vtb70186744, access=READ_EXECUTE, inode="/data/adwh/prod_rls/cd_deposit":hive:hive:drwxrwx--x
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:410)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:264)
	at org.apache.sentry.hdfs.SentryINodeAttributesProvider$SentryPermissionEnforcer.checkPermission(SentryINodeAttributesProvider.java:86)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:194)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1875)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1859)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPathAccess(FSDirectory.java:1809)
	at org.apache.hadoop.hdfs.server.nam

23/10/23 06:44:44 ERROR cluster.YarnClientSchedulerBackend: YARN application has exited unexpectedly with state SUCCEEDED! Check the YARN application logs for more details.
23/10/23 06:46:57 ERROR util.Utils: Uncaught exception in thread YARN application state monitor
org.apache.spark.rpc.RpcTimeoutException: Futures timed out after [120 seconds]. This timeout is controlled by spark.rpc.askTimeout
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:36)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:76)
	at org.apache.spark.scheduler.cluster.CoarseGrainedSchedulerBackend.requestTotalExecutors(CoarseGrainedSchedulerBackend.scala:594)


## Калькулятор маржи

Внутренняя стоимость средств на накопительных счетах (ВССНС) ФЛ в рублях  
устанавливается в виде фиксированной ставки как сумма значения Ключевой  
ставки Банка России и надбавки, равной среднекалендарному значению  
трансфертной ставки в рублях на базе КС на срок 1 год за предшествующие дате  
расчета 1095 дней (с учетом выходных и праздничных дней).  

## Рассмотрим weighted_rate

In [55]:
# data_rates = pd.read_csv('SA_rates_table_v4_prod.csv')

In [56]:
# data_rates.head(3)

In [57]:
# data_rates['weighted_rate_x_balance'] = data_rates['weighted_rate'] * data_rates['balance']

In [58]:
# data_rates.groupby('report_date').sum()['weighted_rate_x_balance'] / data_rates.groupby('report_date').sum()['balance']

In [59]:
# data_rates['report_dt'] = data_rates['report_year'].astype(str) + '-' + data_rates['report_month'].astype(str)

In [ ]:
# data_rates['avg_rate_x_avg_balance'] = data_rates['avg_rate'] * data_rates['avg_balance']

In [ ]:
# data_rates.groupby('report_dt').sum()['avg_rate_x_avg_balance'] / data_rates.groupby('report_dt').sum()['avg_balance']

In [ ]:
# data_rates['avg_weighted_rate_x_avg_balance'] = data_rates['avg_weighted_rate'] * data_rates['avg_balance']

In [60]:
# data_rates.groupby(['report_dt', 'is_vip_or_prv']).sum()['avg_weighted_rate_x_avg_balance'] / data_rates.groupby(['report_dt', 'is_vip_or_prv']).sum()['avg_balance']

## Смотрим расчет по балансу

In [10]:
sa_fact = pd.read_csv('saving_accounts_fact.csv')

In [11]:
sa_fact = sa_fact[(sa_fact['report_dt']>='2023-01-01')&(sa_fact['report_dt']<='2023-08-01')]

In [12]:
sa_fact.head(10)

,report_dt,balance_general,sa_avg_balance_[general]_[mass],sa_avg_balance_[general]_[priv],sa_avg_balance_[general]_[vip],sa_weighted_rate_[general]_[mass],sa_weighted_rate_[general]_[priv],sa_weighted_rate_[general]_[vip]
1,2023-07-31 00:00:00,1.208002e+12,7.893295e+11,3.842425e+11,3.443007e+10,5.578756,5.085335,4.810358
2,2023-06-30 00:00:00,1.166320e+12,7.596419e+11,3.728167e+11,3.386101e+10,5.497468,5.075955,4.733183
3,2023-05-31 00:00:00,1.112985e+12,7.413980e+11,3.403776e+11,3.120987e+10,5.396878,5.044873,4.737019
4,2023-04-30 00:00:00,1.095063e+12,7.248563e+11,3.392324e+11,3.097406e+10,5.305644,5.001618,4.744042
5,2023-03-31 00:00:00,1.083712e+12,7.098920e+11,3.429391e+11,3.088107e+10,5.130050,4.904298,4.714634
6,2023-02-28 00:00:00,1.084683e+12,7.058291e+11,3.417648e+11,3.708912e+10,5.076594,4.871131,4.686045
7,2023-01-31 00:00:00,1.087555e+12,7.029266e+11,3.503515e+11,3.427652e+10,5.028761,4.852665,4.687145


In [13]:
#sa_fact.to_excel('sa_fact.xlsx')

In [14]:
sa_fact['sa_weighted_rate_general'] = (sa_fact['sa_avg_balance_[general]_[mass]'] * sa_fact['sa_weighted_rate_[general]_[mass]'] \
+ sa_fact['sa_avg_balance_[general]_[priv]'] * sa_fact['sa_weighted_rate_[general]_[priv]'] \
+ sa_fact['sa_avg_balance_[general]_[vip]'] * sa_fact['sa_weighted_rate_[general]_[vip]']) / sa_fact['balance_general']

In [15]:
sa_fact['date'] = sa_fact['report_dt'].apply(lambda x: x[:7])

In [16]:
import pip

In [17]:
#pip.main(['install', 'openpyxl'])

In [18]:
import openpyxl

In [19]:
sa_ftp = pd.read_excel('sa_ftp_rates.xlsx')

In [20]:
sa_ftp.head()

,Дата,Валюта,Инструмент,До востребования,Комментарий,Обозначение базиса плавающей ставки,Обозначение инструмента
0,2023-09-18,RUB,Внутренняя стоимость средств на накопительных ...,0.1374,Ставки утверждены Решением ФК от 15/09/2023,Fixed,ODIP_SAV
1,2023-09-18,USD,Внутренняя стоимость средств на накопительных ...,0.0162,Ставки утверждены Решением ФК от 15/09/2023,Fixed,ODIP_SAV
2,2023-09-18,EUR,Внутренняя стоимость средств на накопительных ...,0.0162,Ставки утверждены Решением ФК от 15/09/2023,Fixed,ODIP_SAV
3,2023-08-15,RUB,Внутренняя стоимость средств на накопительных ...,0.1274,Ставки утверждены Решением ФК,Fixed,ODIP_SAV
4,2023-08-15,USD,Внутренняя стоимость средств на накопительных ...,0.0153,Ставки утверждены Решением ФК,Fixed,ODIP_SAV


In [21]:
sa_ftp = sa_ftp[(sa_ftp['Валюта']=='RUB')]

In [22]:
sa_ftp['До востребования'][sa_ftp['Дата']=='2023-07-24'] = 0.085

In [23]:
sa_ftp[sa_ftp['Дата']=='2023-07-24']['До востребования']

6    0.085
Name: До востребования, dtype: float64

In [24]:
sa_ftp['date'] = sa_ftp['Дата'].apply(lambda x: str(x)[:7])

In [25]:
sa_ftp.rename(columns={'До востребования': 'ftp_rate'}, inplace=True)

In [26]:
sa_ftp = sa_ftp[['date', 'ftp_rate']]

In [27]:
sa_fact = sa_fact.merge(sa_ftp, on='date', how='left')

In [28]:
sa_fact['ftp_rate'] = sa_fact['ftp_rate']*100

In [29]:
sa_fact.head()

,report_dt,balance_general,sa_avg_balance_[general]_[mass],sa_avg_balance_[general]_[priv],sa_avg_balance_[general]_[vip],sa_weighted_rate_[general]_[mass],sa_weighted_rate_[general]_[priv],sa_weighted_rate_[general]_[vip],sa_weighted_rate_general,date,ftp_rate
0,2023-07-31 00:00:00,1.208002e+12,7.893295e+11,3.842425e+11,3.443007e+10,5.578756,5.085335,4.810358,5.399908,2023-07,8.50
1,2023-06-30 00:00:00,1.166320e+12,7.596419e+11,3.728167e+11,3.386101e+10,5.497468,5.075955,4.733183,5.340541,2023-06,8.26
2,2023-05-31 00:00:00,1.112985e+12,7.413980e+11,3.403776e+11,3.120987e+10,5.396878,5.044873,4.737019,5.270723,2023-05,NaN
3,2023-04-30 00:00:00,1.095063e+12,7.248563e+11,3.392324e+11,3.097406e+10,5.305644,5.001618,4.744042,5.195577,2023-04,NaN
4,2023-03-31 00:00:00,1.083712e+12,7.098920e+11,3.429391e+11,3.088107e+10,5.130050,4.904298,4.714634,5.046774,2023-03,NaN


In [30]:
sa_fact['sa_weighted_rate_general'] = sa_fact['sa_weighted_rate_[general]_[vip]']

In [64]:
SSV = 0.48
FOR = [4.5, 4.5, 4, 4, 4, 3, 3]
WR = [4.897288112, 4.886817698, 4.869371293, 4.851317322, 4.83055387, 4.829810658, 4.826585737]

In [32]:
sa_fact['ftp_rate'] = sa_fact['ftp_rate'].fillna(method='bfill')

In [65]:
sa_fact['weighted_rate_calc'] = WR

In [62]:
sa_fact['ftp_rate']

0    8.50
1    8.26
2    8.18
3    8.18
4    8.18
5    8.18
6    8.18
Name: ftp_rate, dtype: float64

In [66]:
sa_fact['weighted_rate_calc']

0    4.897288
1    4.886818
2    4.869371
3    4.851317
4    4.830554
5    4.829811
6    4.826586
Name: weighted_rate_calc, dtype: float64

In [35]:
#wr_day = sa_fact['sa_weighted_rate_general']**(1/365)

In [36]:
#((wr_day)**(sa_fact['report_dt'].apply(lambda x: int(x[8:10])))-1)

In [67]:
ftp = sa_fact['ftp_rate'] * (1- pd.Series(FOR) / 100) - SSV - sa_fact['weighted_rate_calc']

In [68]:
1- pd.Series(FOR) / 100

0    0.955
1    0.955
2    0.960
3    0.960
4    0.960
5    0.970
6    0.970
dtype: float64

In [69]:
ftp

0    2.740212
1    2.521482
2    2.503429
3    2.521483
4    2.542246
5    2.624789
6    2.628014
dtype: float64

In [70]:
ftp_month = ((1+ftp/100)**(1/12))

In [71]:
ftp_month

0    1.002255
1    1.002077
2    1.002063
3    1.002077
4    1.002094
5    1.002161
6    1.002164
dtype: float64

In [72]:
ftp_day = ((1+ftp/100)**(1/365))

In [73]:
((ftp_day)**(sa_fact['report_dt'].apply(lambda x: int(x[8:10])))-1)

0    0.002299
1    0.002049
2    0.002102
3    0.002049
4    0.002134
5    0.001990
6    0.002206
dtype: float64

In [74]:
sa_fact['ftp_month'] = ftp_month -1

In [75]:
sa_fact['ftp_month'] = sa_fact['ftp_month'].fillna(method='bfill').head(10)

In [76]:
sa_fact

,report_dt,balance_general,sa_avg_balance_[general]_[mass],sa_avg_balance_[general]_[priv],sa_avg_balance_[general]_[vip],sa_weighted_rate_[general]_[mass],sa_weighted_rate_[general]_[priv],sa_weighted_rate_[general]_[vip],sa_weighted_rate_general,date,ftp_rate,ftp_month,margin_general,margin_mass,margin_priv,margin_vip,weighted_rate_calc
0,2023-07-31 00:00:00,1.208002e+12,7.893295e+11,3.842425e+11,3.443007e+10,5.578756,5.085335,4.810358,4.810358,2023-07,8.50,0.002255,2.863719e+09,1.871204e+09,9.108947e+08,8.162076e+07,4.897288
1,2023-06-30 00:00:00,1.166320e+12,7.596419e+11,3.728167e+11,3.386101e+10,5.497468,5.075955,4.733183,4.733183,2023-06,8.26,0.002077,2.533464e+09,1.650084e+09,8.098277e+08,7.355244e+07,4.886818
2,2023-05-31 00:00:00,1.112985e+12,7.413980e+11,3.403776e+11,3.120987e+10,5.396878,5.044873,4.737019,4.737019,2023-05,8.18,0.002063,2.461988e+09,1.640015e+09,7.529347e+08,6.903803e+07,4.869371
3,2023-04-30 00:00:00,1.095063e+12,7.248563e+11,3.392324e+11,3.097406e+10,5.305644,5.001618,4.744042,4.744042,2023-04,8.18,0.002077,2.337946e+09,1.547560e+09,7.242572e+08,6.612926e+07,4.851317
4,2023-03-31 00:00:00,1.083712e+12,7.098920e+11,3.429391e+11,3.088107e+10,5.130050,4.904298,4.714634,4.714634,2023-03,8.18,0.002094,2.417350e+09,1.583499e+09,7.649669e+08,6.888395e+07,4.830554
5,2023-02-28 00:00:00,1.084683e+12,7.058291e+11,3.417648e+11,3.708912e+10,5.076594,4.871131,4.686045,4.686045,2023-02,8.18,0.002161,2.274744e+09,1.480230e+09,7.167325e+08,7.778149e+07,4.829811
6,2023-01-31 00:00:00,1.087555e+12,7.029266e+11,3.503515e+11,3.427652e+10,5.028761,4.852665,4.687145,4.687145,2023-01,8.18,0.002164,2.524427e+09,1.631630e+09,8.132341e+08,7.956249e+07,4.826586


In [77]:
sa_fact['ftp_month'] = ((ftp_day)**(sa_fact['report_dt'].apply(lambda x: int(x[8:10])))-1)

In [78]:
sa_fact['ftp_month']

0    0.002299
1    0.002049
2    0.002102
3    0.002049
4    0.002134
5    0.001990
6    0.002206
Name: ftp_month, dtype: float64

In [79]:
sa_fact['margin_general'] = sa_fact['ftp_month'] * sa_fact['balance_general']

In [80]:
sa_fact['margin_mass'] = sa_fact['ftp_month'] * sa_fact['sa_avg_balance_[general]_[mass]']
sa_fact['margin_priv'] = sa_fact['ftp_month'] * sa_fact['sa_avg_balance_[general]_[priv]']
sa_fact['margin_vip'] = sa_fact['ftp_month'] * sa_fact['sa_avg_balance_[general]_[vip]']

In [82]:
sa_fact[['report_dt', 'balance_general', 'margin_general', 'margin_priv', 'margin_vip', 'margin_mass', 'weighted_rate_calc']].sort_values(by='report_dt').T

,6,5,4,3,2,1,0
report_dt,2023-01-31 00:00:00,2023-02-28 00:00:00,2023-03-31 00:00:00,2023-04-30 00:00:00,2023-05-31 00:00:00,2023-06-30 00:00:00,2023-07-31 00:00:00
balance_general,1087554620912.959961,1084683033226.77002,1083712146191.5,1095062767847.319946,1112985392767.47998,1166319649864.360107,1208002117514.389893
margin_general,2398727915.835428,2158021895.373902,2313133117.467195,2243619752.264263,2339749462.248781,2389614088.743225,2776744248.614958
margin_priv,772740816.081146,679955333.835812,731987570.052635,695036448.356663,715551349.978634,763845586.779311,883229561.026883
margin_vip,75600836.886325,73790350.247673,65914214.166457,63461222.183216,65610279.293459,69376125.962477,79141821.485811
margin_mass,1550386262.867957,1404276211.290417,1515231333.248104,1485122081.724384,1558587832.976688,1556392376.001436,1814372866.102264
weighted_rate_calc,4.826586,4.829811,4.830554,4.851317,4.869371,4.886818,4.897288


In [51]:
sa_fact[['report_dt', 'balance_general', 'margin_general', 'margin_priv', 'margin_vip', 'margin_mass', 'sa_weighted_rate_general']].sort_values(by='report_dt').T.to_excel('sa_fact_res_v3.xlsx')

In [52]:
sa_fact

,report_dt,balance_general,sa_avg_balance_[general]_[mass],sa_avg_balance_[general]_[priv],sa_avg_balance_[general]_[vip],sa_weighted_rate_[general]_[mass],sa_weighted_rate_[general]_[priv],sa_weighted_rate_[general]_[vip],sa_weighted_rate_general,date,ftp_rate,ftp_month,margin_general,margin_mass,margin_priv,margin_vip
0,2023-07-31 00:00:00,1.208002e+12,7.893295e+11,3.842425e+11,3.443007e+10,5.578756,5.085335,4.810358,4.810358,2023-07,8.50,0.002371,2.863719e+09,1.871204e+09,9.108947e+08,8.162076e+07
1,2023-06-30 00:00:00,1.166320e+12,7.596419e+11,3.728167e+11,3.386101e+10,5.497468,5.075955,4.733183,4.733183,2023-06,8.26,0.002172,2.533464e+09,1.650084e+09,8.098277e+08,7.355244e+07
2,2023-05-31 00:00:00,1.112985e+12,7.413980e+11,3.403776e+11,3.120987e+10,5.396878,5.044873,4.737019,4.737019,2023-05,8.18,0.002212,2.461988e+09,1.640015e+09,7.529347e+08,6.903803e+07
3,2023-04-30 00:00:00,1.095063e+12,7.248563e+11,3.392324e+11,3.097406e+10,5.305644,5.001618,4.744042,4.744042,2023-04,8.18,0.002135,2.337946e+09,1.547560e+09,7.242572e+08,6.612926e+07
4,2023-03-31 00:00:00,1.083712e+12,7.098920e+11,3.429391e+11,3.088107e+10,5.130050,4.904298,4.714634,4.714634,2023-03,8.18,0.002231,2.417350e+09,1.583499e+09,7.649669e+08,6.888395e+07
5,2023-02-28 00:00:00,1.084683e+12,7.058291e+11,3.417648e+11,3.708912e+10,5.076594,4.871131,4.686045,4.686045,2023-02,8.18,0.002097,2.274744e+09,1.480230e+09,7.167325e+08,7.778149e+07
6,2023-01-31 00:00:00,1.087555e+12,7.029266e+11,3.503515e+11,3.427652e+10,5.028761,4.852665,4.687145,4.687145,2023-01,8.18,0.002321,2.524427e+09,1.631630e+09,8.132341e+08,7.956249e+07


Идея - трансфертная ставка идет на момент открытия. Попробовать рассчитать все это дело.

In [53]:
sa_fact['perc'] = sa_fact['margin'] / sa_fact['balance_general']  * 365 / 30.3 * 100

KeyError: 'margin'

In [54]:
sa_fact['perc']

KeyError: 'perc'

In [ ]:
sa_fact